# Проект по SQL
    
Данный проект разделим на несколько частей:
    
### Часть 1. Подключение к базе данных
    
### Часть 2. Изучение общей информации
    
### Часть 3. Решение задач
    
### Часть 4. Общий вывод


# 1. Подключение к базе данных
<a id='section1'></a>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

# 2. Изучение общей информации
<a id='section2'></a>

### 2.1. Таблица books:

In [2]:
query = '''

SELECT 
    * 
FROM 
    books
LIMIT 5

'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица books содержит данные о книгах и состоит из 6 столбцов.

### 2.2. Таблица authors:

In [3]:
query = '''

SELECT 
    * 
FROM 
    authors
LIMIT 5

'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица authors содержит данные об авторах и состоит из 2 столбцов.

### 2.3. Таблица publishers:

In [4]:
query = '''

SELECT 
    * 
FROM 
    publishers
LIMIT 5

'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица publishers содержит данные об издательствах и состоит из2 столбцов.

### 2.4. Таблица ratings:

In [5]:
query = '''

SELECT 
    * 
FROM 
    ratings
LIMIT 5

'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица ratings содержит данные о пользовательских оценках книг и состоит из 4 столбцов.

### 2.5. Таблица reviews:

In [6]:
query = '''

SELECT 
    * 
FROM 
    reviews
LIMIT 5

'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица reviews содержит данные о пользовательских обзорах на книги и состоит из 3 столбцов.

# 3. Решение задач 
<a id='section3'></a>

### 3.1. Cколько книг вышло после 1 января 2000 года?

In [7]:
query='''
SELECT COUNT(book_id) AS "books_count"
FROM books
WHERE publication_date>'2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,books_count
0,819


С 1 января 2000 года вышло 819 книг.


### 3.2. Для каждой книги посчитайте количество обзоров и среднюю оценку


In [8]:
query = '''
SELECT 
    books.title, 
    COUNT(distinct reviews.review_id) AS reviews_cnt, 
    AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY 
    books.book_id
ORDER BY 
    COUNT(reviews.review_id) DESC
LIMIT 10
'''
pd.io.sql.read_sql(query, con = engine)

,title,reviews_cnt,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
5,Angels & Demons (Robert Langdon #1),5,3.678571
6,Harry Potter and the Order of the Phoenix (Har...,5,4.186667
7,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
8,Animal Farm,5,3.729730
9,The Fellowship of the Ring (The Lord of the Ri...,5,4.391892


В данной таблице представлены названия книг, количество обзоров и средний рейтинг.

### 3.3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [9]:
query = '''
SELECT
    publishers.publisher,
    COUNT(books.book_id) AS count_books
FROM
    books    
LEFT JOIN publishers ON  publishers.publisher_id = books.publisher_id 
WHERE 
    books.num_pages > 50
GROUP BY
    publishers.publisher
ORDER BY 
    COUNT(books.book_id) DESC
LIMIT 10
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


Наибольшее число книг выпустило издательство Penguin Books (42 книги), далее следуют издательства: Vintage(31), Grand Central Publishing (25), Penguin Classics (24), Ballantine Books(19), Bantam(19) и другие (в таблице).

### 3.4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [10]:

query='''
SELECT
    authors.author,
    AVG(SUB.average_rating) AS avg_rating
FROM
    (SELECT
        book_id AS book_id,
        COUNT(rating_id) AS rating_counts,
        AVG(rating) AS average_rating
    FROM
        ratings
    GROUP BY 
        book_id
    HAVING 
        COUNT(rating_id) > 50) AS sub
INNER JOIN books ON books.book_id = SUB.book_id
INNER JOIN authors ON authors.author_id = books.author_id
GROUP BY authors.author
ORDER BY avg_rating DESC
LIMIT 1

'''
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг J.K. Rowling/Mary GrandPré - рейтинг 4.28. 

### 3.5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [11]:
query = '''
SELECT
    AVG(sub.review_cnt) AS avg_review_cnt
FROM
    (SELECT
        COUNT(text) AS review_cnt
    FROM 
        reviews
    WHERE username IN
        (SELECT username
        FROM 
            ratings
        GROUP BY
            username
        HAVING
            COUNT(ratings) > 50)
    GROUP BY 
            username
    ) AS sub 
'''
pd.io.sql.read_sql(query, con = engine)

,avg_review_cnt
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.3. 

# 4. Общий вывод по задачам
<a id='section4'></a>

1. С 1 января 2000 года вышло 819 книг.
1. Была построена таблица с названием книг, количеством обзоров и средним рейтингом.
1. Наибольшее число книг выпустило издательство Penguin Books (42 книги), далее следуют издательства: Vintage(31), Grand Central Publishing (25), Penguin Classics (24), Ballantine Books(19), Bantam(19) и другие (в таблице).
1. Автор с самой высокой средней оценкой книг J.K. Rowling/Mary GrandPré - рейтинг 4.28.
1. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.3.